In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from collections import Counter
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
train_anno = pd.read_csv("../data/eBay/metadata/train.csv").to_dict()
val_anno = pd.read_csv("../data/eBay/metadata/val.csv").to_dict()

In [ ]:
train_labels = list(train_anno["LEAF_CATEG_ID"].values())
val_labels = list(val_anno["LEAF_CATEG_ID"].values())

In [ ]:
train_paths = list(train_anno["IMAGE_PATH"].values())
val_paths = list(val_anno["IMAGE_PATH"].values())
paths = train_paths + val_paths

In [ ]:
train_texts = list(train_anno["AUCT_TITL"].values())
val_texts = list(val_anno["AUCT_TITL"].values())
texts = train_texts + val_texts

In [ ]:
len(train_labels), len(val_labels)

In [ ]:
labels = train_labels + val_labels

In [ ]:
labels = np.array(labels)

In [ ]:
image_feats = np.load("convnext384_feats.npy")
text_feats = np.load("roberta_avg_feats.npy")

In [ ]:
label_freq = Counter(labels)

In [ ]:
label_freq.most_common()

In [ ]:
class_index = np.where(labels == 408)[0]

In [ ]:
selected_image_feats = image_feats[class_index]
selected_text_feats = text_feats[class_index]

In [ ]:
def dbscan(eps, feats):
    cluster = DBSCAN(eps=eps, min_samples=1, metric='cosine', n_jobs=-1)
    return cluster.fit_predict(feats)

def get_large_cluster(predict_labels, threshold=10):
    large_cluster = []
    for cluster_label, freq in Counter(predict_labels).most_common():
        if freq > threshold:
            large_cluster.append(cluster_label)
        else:
            break
    return large_cluster

def hierarchical_cluster(maintain_labels, predict_labels, feats, eps=0.5):
    large_cluster = get_large_cluster(predict_labels)
    for c in large_cluster:
        cluster_index = np.where(predict_labels == c)[0]
        new_feats = feats[cluster_index]
        new_predict_labels = dbscan(eps, new_feats)
        new_maintain_labels = [""] * len(cluster_index)
        for i, idx in enumerate(cluster_index):
            new_maintain_labels[i] += maintain_labels[idx] + "." + str(new_predict_labels[i])
        if eps > 0.01:
            new_maintain_labels = hierarchical_cluster(new_maintain_labels, new_predict_labels, new_feats, eps-0.01)
        for i, idx in enumerate(cluster_index):
            maintain_labels[idx] = new_maintain_labels[i]
    return maintain_labels
    
def do_cluster(selected_image_feats, eps=0.5):
    predict_labels = dbscan(eps, selected_image_feats)
    maintain_labels = [str(x) for x in predict_labels]
    maintain_labels = hierarchical_cluster(maintain_labels, predict_labels, selected_image_feats, eps-0.1)
    label_set = set(maintain_labels)
    reassign_dict = dict(zip(set(label_set), range(len(label_set))))
    predict_labels = [reassign_dict[x] for x in maintain_labels]
    return np.array(predict_labels)

In [ ]:
print(max(predict_labels))
Counter(predict_labels).most_common()

In [ ]:
show_predict_label = 55
show_index_index = np.where(predict_labels == show_predict_label)[0]
show_index = class_index[show_index_index]

for idx in show_index[:10]:
    print(texts[idx], "=========", idx)
    image = Image.open("../data/eBay/Images/" + paths[idx])
    plt.figure()
    plt.imshow(image)

In [ ]:
all_labels = [""] * len(image_feats)
for class_name, freq in tqdm(label_freq.most_common()):
    class_index = np.where(labels == class_name)[0]
    selected_image_feats = image_feats[class_index]
    predict_labels = do_cluster(selected_image_feats)
    for i, idx in enumerate(class_index):
        all_labels[idx] += str(class_name) + "." + str(predict_labels[i])

In [ ]:
label_set = set(all_labels)
reassign_dict = dict(zip(set(label_set), range(len(label_set))))
reassign_all_labels = np.array([reassign_dict[x] for x in all_labels])

In [ ]:
Counter(reassign_all_labels).most_common()

In [ ]:
show_predict_label = 603315
show_index = np.where(reassign_all_labels == show_predict_label)[0]

for idx in show_index:
    print(texts[idx], "=========", idx)
    image = Image.open("../data/eBay/Images/" + paths[idx])
    plt.figure()
    plt.imshow(image)

In [ ]:
all_labels_freq = Counter(reassign_all_labels).most_common()
all_freq = [x[1] for x in all_labels_freq]

In [ ]:
Counter(all_freq).most_common()

In [ ]:
reassign_train_labels = reassign_all_labels[:len(train_labels)]
reassign_val_labels = reassign_all_labels[len(train_labels):]

In [ ]:
len(reassign_train_labels), len(reassign_val_labels)

In [ ]:
np.save("pseudo_train_ids.npy", reassign_train_labels)
np.save("pseudo_val_ids.npy", reassign_val_labels)